In [39]:
import pandas as pd
import os
import numpy as np

import fiona # Needed for geopandas to run
import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Point, Polygon # also needed

In [2]:
os.getcwd() + "\\zillow\\zillow_med_price"


'C:\\Users\\mwaugh\\github\\data_bootcamp_spring_2019\\mapping'

In [5]:
df = pd.read_csv(os.getcwd() + "\\zillow\\zillow_med_price.csv",encoding = "ISO-8859-1")

In [6]:
df = df.melt(id_vars = ["RegionID","RegionName","City","State","Metro","CountyName","SizeRank"
])

In [27]:
nyc_price = df[(df.City == "New York") & (df.variable == "2017-06")].copy()

In [28]:
nyc_price.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,variable,value
3756914,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2017-06,1317.0
3756916,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2017-06,1731.0
3756922,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,9,2017-06,1426.0
3756923,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,10,2017-06,246.0
3756924,62037,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,2017-06,509.0


In [29]:
nyc_price["ZIPCODE"] = nyc_price.RegionName.astype(int)

In [19]:
cwd = os.getcwd()

regions_shape = cwd + "\\shapefile\\ZIP_CODE_040114.shx"

regions_shape

nyc_map = gpd.read_file(regions_shape)

nyc_map.ZIPCODE = nyc_map.ZIPCODE.astype(int) # we want these to look like numbers


In [30]:
nyc_map  = nyc_map.merge(nyc_price, on='ZIPCODE', how = "left", indicator = True)

In [40]:
nyc_map.variable.replace(np.nan,0.0, inplace = True)

In [41]:
nyc_map.tail()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,...,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,variable,value,_merge
258,10310,0,Staten Island,25003.0,5.346328e+07,NY,Richmond,36,085,http://www.usps.com/,...,61786.0,10310.0,New York,NY,New York-Newark-Jersey City,Richmond County,5729.0,2017-06,315.0,both
259,11693,0,Far Rockaway,11052.0,3.497516e+06,NY,Kings,36,047,http://www.usps.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,left_only
260,11249,0,Brooklyn,28481.0,1.777221e+07,NY,Kings,36,047,http://www.usps.com/,...,62058.0,11249.0,New York,NY,New York-Newark-Jersey City,Kings County,1717.0,2017-06,1202.0,both
261,10162,1,New York,0.0,2.103489e+04,NY,New York,36,061,http://www.usps.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,left_only
262,10119,1,New York,0.0,1.263930e+05,NY,New York,36,061,http://www.usps.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,left_only
